In [ ]:
'''this generates the xj for corresponding xi. use different models to experiment. device=0 uses gpu if there is any.'''

from transformers import pipeline
import pandas as pd


generator = pipeline('text-generation', model='gpt2')
# generator = pipeline('text-generation', model='bigscience/bloom-1b7', device=0)
# generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B', device=0)

df = pd.read_csv("xi.csv")
truncated_texts = df['truncated_text'].tolist()

completions = []
for truncated_text in truncated_texts:
    completion = generator(truncated_text, max_length=50, num_return_sequences=1)[0]['generated_text']
    completions.append(completion)

df['gpt2_completion'] = completions
df.to_csv("completions_gpt2.csv", index=False)



In [3]:
'''this combines and shuffles the dataset.'''
import pandas as pd

df_gpt2 = pd.read_csv("completions_gpt2.csv")
df_gpt_neo = pd.read_csv("completions_gpt_neo.csv")
df_bloom = pd.read_csv("completions_bloom_1b7.csv")

df_gpt2['model'] = 'gpt2'
df_gpt_neo['model'] = 'gpt_neo'
df_bloom['model'] = 'bloom'

df_gpt2.rename(columns={'gpt2_completion': 'completion'}, inplace=True)
df_gpt_neo.rename(columns={'gpt_neo_completion': 'completion'}, inplace=True)
df_bloom.rename(columns={'bloom_completion': 'completion'}, inplace=True)

combined_df = pd.concat([df_gpt2[['completion', 'model']],
                         df_gpt_neo[['completion', 'model']],
                         df_bloom[['completion', 'model']]], ignore_index=True)

combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

combined_df.to_csv("combined_completions_normal.csv", index=False)
